In [ ]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict
from concurrent.futures import ProcessPoolExecutor
from tqdm.auto import tqdm

In [ ]:
path = "/root/robocup2d_data_processed2"
data_list = []
for root, dirs, files in tqdm(
    os.walk(path), total=len([None for _, _, _ in os.walk(path)])
):
    for file in tqdm(files, leave=False):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(root, file))
            df = df.where(pd.notnull(df), None)
            int_cols = df.select_dtypes(include=["int64"]).columns
            for col in int_cols:
                df[col] = df[col].astype("float64")
            data_list.append(df)

In [ ]:
len(data_list)

In [ ]:
# pd.set_option('display.max_columns', 1000)

data_list[0].dtypes.unique()

# for i in data_list[0].dtypes:
#     print(i)

In [ ]:
def generator(data):
    def _generator():
        for d in tqdm(data):
            yield d

    return _generator


dataset = Dataset.from_generator(generator(data_list))
dataset

In [ ]:
dataset.push_to_hub("<Repository Name>")